In [86]:
#%%writefile main_in_periodo.py

import pandas as pd
import numpy as np
import boto3
# import seaborn as sns
# import matplotlib.pyplot as plt
import statsmodels.api as sm
# from scipy import stats
from datetime import datetime
from calendar import monthrange
import calendar as c
import boto3


def isdir_s3(bucket, key):
    objs = list(bucket.objects.filter(Prefix=key))
#     print('I am the master_dir')
    return len(objs)

def costruzione_file_da_mensile(source, anno_inizio, anno_fine,mese_inizio,mese_fine):
    bucketname = 'zus-prod-s3'
    s3 = boto3.resource('s3')
#     source = 'preprocessato/sistema/temperatura/epson/temperatura'
    my_bucket = s3.Bucket(bucketname)

    df = pd.DataFrame()
    
    mese_inizio = int(mese_inizio)
    mese_fine = int(mese_fine)
    
    anno_inizio = int(anno_inizio)
    anno_fine = int(anno_fine)
    
    #pd.date_range(str(2000)+'-'+str(11),str(2001)+'-'+str(3).rjust(2,'0'),freq='MS').strftime('%m').tolist()
    #pd.date_range(str(2000)+'-'+str(11)+'-01',str(2001)+'-'+str(3).rjust(2,'0')+'-01',freq='MS').strftime('%m').tolist()
    mesi = pd.date_range(str(anno_inizio)+'-'+str(mese_inizio),str(anno_inizio+1)+'-'+str(mese_fine).rjust(2,'0'),freq='MS').strftime('%m').tolist()
    stop_index = mesi.index('12')
    anni = np.arange(anno_inizio,anno_fine+1)

    for a in anni:
        for m in mesi:
            if ((a==anni[0]) & (mesi.index(m)>stop_index)):
                print(str(a)+str(m))
            else:
                if isdir_s3(my_bucket,source + '/' +str(a)+str(m).ljust(2,'0'))>0:
                    df = df.append(pd.read_csv('s3://'+bucketname+'/'+source+'/'+str(a)+str(m).ljust(2,'0')+'/epson_best.csv'))
#     annomesi = pd.date_range(inizio, fine,freq='MS').strftime('%Y%m').tolist()
#     for am in annomesi:
#         if isdir_s3(my_bucket,source + '/' +str(am))>0:
#             df = df.append(pd.read_csv('s3://'+bucketname+source+'/'+str(am)+'/epson_best.csv'))
    return df

def lettura_temp_norm(df,tipo_colonna):
    #df = pd.read_csv(file_t_norm)#,encoding='utf-16') 's3://zus-prod-s3/'+
#     df = costruzione_file_da_mensile(file_t_norm,inizio,fine)
#     df = df[['DATA','PROVINCIA','TEMPERATURA_MAX','TEMPERATURA_MIN']]
#     df.rename(columns={'DATA':'date','PROVINCIA':'provincia','TEMPERATURA_MIN':'temp_min','TEMPERATURA_MAX':'temp_max'},inplace=True)
    #df.rename(columns={'TIME - Date':'date','Codice Provincia':'provincia','Temperature Min':'temp_min','Temperature Max':'temp_max'},inplace=True)
    
    df = df[['DATA',tipo_colonna,'TEMPERATURA_MAX','TEMPERATURA_MIN']]
    
    df.rename(columns={'DATA':'date',tipo_colonna.upper():tipo_colonna.lower(),'TEMPERATURA_MIN':'temp_min','TEMPERATURA_MAX':'temp_max'},inplace=True)
    tipo_colonna = tipo_colonna.lower()
    df = df[~df[tipo_colonna].isnull()]
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
    #df = df[(df['date']>=inizio) & (df['date']<=fine)]
    df['T_mean'] = df[['temp_min', 'temp_max']].mean(axis=1)
#     df.drop('Desc_Provincia',axis=1,inplace=True)
#     df.info()

    df = df.assign(
        dayofyear=lambda x: (x.date + pd.DateOffset(days=92)).dt.dayofyear,
        year=lambda x: (x.date + pd.DateOffset(days=92)).dt.year,
    )
#     print(df)
    df['monthyear'] = (df.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df.year).astype('str')
    df['monthday'] = df['date'].dt.month.astype('str').str.pad(2, side='left', fillchar='0') + '-' + df['date'].dt.day.astype('str').str.pad(2, side='left', fillchar='0')
    return df

def lettura_osservatorio(file_oss,anno_inizio,anno_fine):
    df_oss = pd.read_csv(file_oss)

    df_oss.rename(columns={'Cd Oss':'codice_oss','T Oss':'T_oss','Data':'date'},inplace=True)

    df_oss['T_oss'] = df_oss.apply(lambda x: 18 if x['T_oss']==0 else 18-x['T_oss'],axis=1)
    
    df_oss['date'] = df_oss['date'].astype('str')
    df_oss['date'] = pd.to_datetime(df_oss['date'],format='%Y-%m-%d', exact=False)
    df_oss = df_oss[(df_oss['date'].dt.year>=anno_inizio) & (df_oss['date'].dt.year<=anno_fine)]
    df_oss = df_oss.assign(
#         dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    df_oss.drop('Oss',axis=1,inplace=True)
    df_oss['monthyear'] = (df_oss.date.dt.month).astype('str').str.pad(2, side='left', fillchar='0')+(df_oss.year).astype('str')
    return df_oss

def temp_best_col_in_periodo(df,mese_inizio,mese_fine,col,tipo_colonna):
    tipo_colonna = tipo_colonna.lower()
    if mese_inizio <= mese_fine:
        date_value = pd.to_datetime('2020-'+str(mese_fine)+'-01',format='%Y-%m-%d')
        date_value = date_value.replace(day = monthrange(date_value.year, date_value.month)[1])
        count_days = ((date_value - pd.to_datetime('2020-'+str(mese_inizio)+'-01',format='%Y-%m-%d')).days)*df[tipo_colonna.lower()].nunique()
        tmp = df[(df['date'].dt.month>=mese_inizio) & (df['date'].dt.month<=mese_fine)]
#         print(count_days)
        prov = tmp[['year','T_'+col]].groupby('year').count().reset_index()
        prov = prov[prov['T_'+col]>=count_days]
        years_ok = prov.year.unique()
        tmp = tmp[tmp['year'].isin(years_ok)]
        tmp = tmp.loc[~((tmp['date'].dt.month == 2) & (tmp['date'].dt.day == 29))]
        
        print("Individuazione curve di temperatura minima o massima")
        province = tmp.provincia.unique()
        final = pd.DataFrame()
        for prov in province:
            tmp_p = tmp[tmp[tipo_colonna]==prov]
            sums = tmp_p[['year','T_'+col]].groupby('year').sum().reset_index()
            if col=='max':
                year = sums[sums['T_max']==max(sums['T_max'])].year.unique()[0]
                to_drop = 'T_min'
            else:
                year = sums[sums['T_min']==min(sums['T_min'])].year.unique()[0]
                to_drop = 'T_max'
            temp_best_in_period = tmp_p[tmp_p.date.dt.year == year]
            final = final.append(temp_best_in_period)
    else:
        date_value = pd.to_datetime('2021-'+str(mese_fine)+'-01',format='%Y-%m-%d')
        date_value = date_value.replace(day = monthrange(date_value.year, date_value.month)[1])
        count_days = ((date_value - pd.to_datetime('2020-'+str(mese_inizio)+'-01',format='%Y-%m-%d')).days)*df[tipo_colonna].nunique()
        df['to_groupby'] = 0

        df['to_groupby'] = df.apply(lambda x: x.date.year+1 if x.date.month>=mese_inizio else x.date.year,axis=1)
        tmp = df[((df['date'].dt.month>=mese_inizio) | (df['date'].dt.month<=mese_fine))]

        prov = tmp[['to_groupby','T_'+col]].groupby('to_groupby').count().reset_index()
        prov = prov[prov['T_'+col]>=count_days]
        years_ok = prov.to_groupby.unique()
        tmp = tmp[tmp['to_groupby'].isin(years_ok)]
        tmp = tmp.loc[~((tmp['date'].dt.month == 2) & (tmp['date'].dt.day == 29))]
        
        print("Individuazione curve di temperatura minima o massima")
        tmp['monthyear'] = pd.to_datetime(tmp['monthyear'],format='%m%Y')
        province = tmp[tipo_colonna].unique()
        final = pd.DataFrame()
        
        for prov in province:
            tmp_p = tmp[tmp[tipo_colonna]==prov]
            sums = tmp_p[['to_groupby','T_'+col]].groupby('to_groupby').sum().reset_index()
            if col=='max':
                year = sums[sums['T_max']==max(sums['T_max'])].to_groupby.unique()[0]
                to_drop = 'T_min'
            else:
                year = sums[sums['T_min']==min(sums['T_min'])].to_groupby.unique()[0]
                to_drop = 'T_max'
            temp_best_in_period = tmp_p[tmp_p.to_groupby == year]
            final = final.append(temp_best_in_period)
    final.drop(to_drop,axis=1,inplace=True)
    return final

def temp_best_col_in_periodo_oss(df,mese_inizio,mese_fine,col):
    if mese_inizio < mese_fine:
        date_value = pd.to_datetime('2020-'+str(mese_fine)+'-01',format='%Y-%m-%d')
        date_value = date_value.replace(day = monthrange(date_value.year, date_value.month)[1])
        count_days = ((date_value- pd.to_datetime('2020-'+str(mese_inizio)+'-01',format='%Y-%m-%d')).days)*df.codice_oss.nunique()
        tmp = df[(df['date'].dt.month>=mese_inizio) & (df['date'].dt.month<=mese_fine)]
        
        prov = tmp[['year','T_oss']].groupby('year').count().reset_index()
        prov = prov[prov['T_oss']>=count_days]
        years_ok = prov.year.unique()
        tmp = tmp[tmp['year'].isin(years_ok)]
        tmp = tmp.loc[~((tmp['date'].dt.month == 2) & (tmp['date'].dt.day == 29))]
        
        print("Individuazione curve di temperatura minima o massima")
        osservatori = tmp.codice_oss.unique()
        final = pd.DataFrame()
        for oss in osservatori:
            tmp_o = tmp[tmp['codice_oss']==oss]
            sums = tmp_o[['year','T_oss']].groupby('year').sum().reset_index()
            if col=='max':
                year = sums[sums['T_oss']==max(sums['T_oss'])].year.unique()[0]
            else:
                year = sums[sums['T_oss']==min(sums['T_oss'])].year.unique()[0]
            temp_best_in_period = tmp_o[tmp_o.date.dt.year == year]
            final = final.append(temp_best_in_period)
    else:
        date_value = pd.to_datetime('2021-'+str(mese_fine)+'-01',format='%Y-%m-%d')
        date_value = date_value.replace(day = monthrange(date_value.year, date_value.month)[1])
        count_days = ((date_value - pd.to_datetime('2020-'+str(mese_inizio)+'-01',format='%Y-%m-%d')).days)*df.codice_oss.nunique()
        df['to_groupby'] = 0
        df['to_groupby'] = df.apply(lambda x: x.date.year+1 if x.date.month>=mese_inizio else x.date.year,axis=1)
        tmp = df[((df['date'].dt.month>=mese_inizio) | (df['date'].dt.month<=mese_fine))]
        tmp['monthyear'] = pd.to_datetime(tmp['monthyear'],format='%m%Y')
        tmp = tmp.loc[~((tmp['date'].dt.month == 2) & (tmp['date'].dt.day == 29))]

        prov = tmp[['to_groupby','T_oss']].groupby('to_groupby').count().reset_index()
        prov = prov[prov['T_oss']>=count_days]
        years_ok = prov.to_groupby.unique()
        tmp = tmp[tmp['to_groupby'].isin(years_ok)]

        print("Individuazione curve di temperatura minima o massima")
        osservatori = tmp.codice_oss.unique()
        final = pd.DataFrame()
        for oss in osservatori:
            tmp_o = tmp[tmp['codice_oss']==oss]
            sums = tmp_o[['to_groupby','T_oss']].groupby('to_groupby').sum().reset_index()
            if col=='max':
                year = sums[sums['T_oss']==max(sums['T_oss'])].to_groupby.unique()[0]
            else:
                year = sums[sums['T_oss']==min(sums['T_oss'])].to_groupby.unique()[0]
            temp_best_in_period = tmp_o[tmp_o.to_groupby == year]
            final = final.append(temp_best_in_period)
        
    return final

def individuazione_min_max_periodo_norm(file_t_norm,mese_inizio,mese_fine,anno_inizio,anno_fine,col,path_to_output,my_bucket,idrun_f,now,inizio,fine,tipo):

    print("Lettura file temperature per " + tipo + '.')
    if tipo == 'prov':
        tipo_colonna = 'PROVINCIA'
    else:
        tipo_colonna = 'OSSERVATORIO'
        
    tipo_colonna = str(tipo_colonna)
    
    #df = costruzione_file_da_mensile(file_input,anno_inizio,anno_fine,tipo_colonna)

        
    df = lettura_temp_norm(file_t_norm,tipo_colonna)
    tipo_colonna = tipo_colonna.lower()

    print("Numero"+ str(tipo) +" presenti: ", df[tipo_colonna].nunique())

    df.drop(['T_mean'],axis=1,inplace=True)
    df.rename(columns={'temp_max':'T_max','temp_min':'T_min'},inplace=True)

    nome_output = str(mese_inizio) + '_' + str(mese_fine) + '_' + str(anno_inizio) + '_' + str(anno_fine)

    temp_best_period = temp_best_col_in_periodo(df,mese_inizio,mese_fine,col,tipo_colonna)
    temp_best_period.drop(['year','monthyear'],inplace=True,axis=1)
    
    temp_best_period['mese'] = temp_best_period['date'].dt.month
    temp_best_period['giorno'] = temp_best_period['date'].dt.day
    
    temp_best_period = temp_best_period.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    temp_best_period['dayofyear'] = temp_best_period.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    temp_best_period.drop('to_groupby',axis=1,inplace=True)
    temp_best_period.to_csv(path_to_output+col+'_periodo_prov/'+idrun_f+'/'+col+'_periodo.csv',index=False)
    
    path_output_metadati = path_to_output.replace('s3://zus-prod-s3/','')
    idrun = path_output_metadati+col+'_periodo_'+str(tipo)+'/'+idrun_f+'/'+col+'_periodo.csv'
    metadatati = pd.DataFrame(data={'CATEGORIA':['TEMPERATURA_NORM']*10,'FLUSSO':[col.upper()+'_PERIODO_'+tipo.upper()]*10,'TIMESTAMP':[now]*10,'ID':['TEMPERATURA_NORM_'+col.upper()+'_PERIODO_PROV_'+tipo.upper()]*10,'PATH': [idrun]*10,'NOME_PARAMETRO':['FILE_INPUT','DATA_INIZIO','DATA_FINE','MESE_INIZIO','MESE_FINE','ANNO_INIZIO','ANNO_FINE','COLONNA','PATH_TO_OUTPUT','TIPO'],'VALORE_PARAMETRO':[file_t_norm,inizio,fine,mese_inizio,mese_fine,anno_inizio,anno_fine,col,path_to_output+col+'_periodo_'+tipo+'/best/'+col+'_periodo_'+nome_output+'.csv',tipo]})
    #metadatati.to_csv('./metadati/min_max_prov.csv',index=False)

    metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/'+col+'_periodo_'+tipo+'/'+idrun_f+'/metadati.csv',index=False)

def individuazione_min_max_periodo_oss(file_oss,mese_inizio_oss,mese_fine_oss,anno_inizio,anno_fine,col_oss,path_to_output,my_bucket,idrun_f,now,inizio,fine,tip):

    print("Lettura file temperature per osservatorio.")
    df_oss = lettura_osservatorio(file_oss,anno_inizio,anno_fine)
    
    print("Numero osservatori presenti: ", df_oss['codice_oss'].nunique())
    nome_output = str(mese_inizio_oss) + '_' + str(mese_fine_oss) + '_' + str(anno_inizio) + '_' + str(anno_fine)
    
    temp_best_period_oss = temp_best_col_in_periodo_oss(df_oss,mese_inizio_oss,mese_fine_oss,col_oss)

    temp_best_period_oss.drop(['year','monthyear'],inplace=True,axis=1)

    temp_best_period_oss['mese'] = temp_best_period_oss['date'].dt.month
    temp_best_period_oss['giorno'] = temp_best_period_oss['date'].dt.day
    
    temp_best_period_oss = temp_best_period_oss.assign(
        dayofyear=lambda x: (x.date).dt.dayofyear, # + pd.DateOffset(days=92)
        year=lambda x: (x.date).dt.year, # + pd.DateOffset(days=92)
    )
    temp_best_period_oss['dayofyear'] = temp_best_period_oss.apply(lambda x: x['dayofyear']+1 if (c.isleap(x['year'])==False and x['date'].month>=3 and x['date'].month<=12) else x['dayofyear'],axis=1)
    temp_best_period_oss.drop('to_groupby',axis=1,inplace=True)


    temp_best_period_oss.to_csv(path_to_output+col_oss+'_periodo_oss/'+idrun_f+'/'+col_oss+'_periodo.csv',index=False)
    
    path_output_metadati = path_to_output.replace('s3://zus-prod-s3/','')
    idrun = path_output_metadati+col_oss+'_periodo_oss/'+idrun_f+'/'+col_oss+'_periodo.csv'
    metadatati = pd.DataFrame(data={'CATEGORIA':['TEMPERATURA_NORM']*9,'FLUSSO':[col_oss.upper()+'_PERIODO_OSS']*9,'TIMESTAMP':[now]*9,'ID':['TEMPERATURA_NORM_'+col_oss.upper()+'_PERIODO_OSS_'+now]*9,'PATH':[idrun]*9,'NOME_PARAMETRO':['FILE_INPUT','DATA_INIZIO','DATA_FINE','MESE_INIZIO','MESE_FINE','ANNO_INIZIO','ANNO_FINE','COLONNA','PATH_TO_OUTPUT'],'VALORE_PARAMETRO':[file_oss,inizio,fine,mese_inizio_oss,mese_fine_oss,anno_inizio,anno_fine,col_oss,path_to_output+col_oss+'_periodo_prov/best/'+col_oss+'_periodo_'+nome_output+'.csv']})
    #metadatati.to_csv('./metadati/min_max_oss.csv',index=False)

    metadatati.to_csv('s3://'+ my_bucket +'/metadati/sistema/temperatura_norm/zeus/metadati/'+col_oss+'_periodo_oss/'+idrun_f+'/metadati.csv',index=False)

def main_in_periodo(file_input,mese_inizio,mese_fine,anno_inizio,anno_fine,col,path_output,tipo):
    print('Funzione dedicata all\'individuazione della temperatura massima e minima nel periodo definito.')
    print('\n')
    print(mese_inizio)
    print(mese_fine)
    
    bucket = 'zus-prod-s3'
    
    #file_input = 's3://' +bucket + '/' + file_input
    path_output = 's3://' +bucket + '/'  + path_output
    
    now = str(datetime.now())[0:-7].replace('-','').replace(' ','').replace(':','')
    inizio =  str(anno_inizio)+str(mese_inizio)
    fine = str(anno_fine)+str(mese_fine)
    
    idrun = str(anno_inizio)+str(mese_inizio) + "_" + str(anno_fine)+str(mese_fine) + str(datetime.now())[0:-7].replace('-','').replace(' ','').replace(':','')
    
    anno_inizio = int(anno_inizio)
    anno_fine = int(anno_fine)
    
    print('costruzione da mensile')
    df = costruzione_file_da_mensile(file_input,anno_inizio,anno_fine,mese_inizio,mese_fine)
    
    #df = pd.read_csv(file_input)#,encoding='utf-16')
    #if 'Codice Provincia' in df.columns:
    print('Individuazione curva con temperatura normale min/max nel periodo definito')
    individuazione_min_max_periodo_norm(df,mese_inizio,mese_fine,anno_inizio,anno_fine,col,path_output,bucket,idrun,now,inizio,fine,tipo)

#         print('\n')
#     else:
#         print('Individuazione curva con temperatura osservatorio min/max nel periodo definito')
#         individuazione_min_max_periodo_oss(file_input,mese_inizio,mese_fine,anno_inizio,anno_fine,col,path_output,bucket,idrun,now,inizio,fine,tipo)
        
#         print('\n')
    print("Procedura terminata.")



In [85]:
main_in_periodo('preprocessato/sistema/temperatura/epson/temperatura',11,3,2000,2020,'max','epson_centralizzato/','oss')
##### NUOVO PATH: 'preprocessato/sistema/temperatura/epson/temperatura'

Funzione dedicata all'individuazione della temperatura massima e minima nel periodo definito.


11
3
costruzione da mensile
200001
200002
200003
     PROVINCIA  AREA      CAPOLUOGO  CIELO COD_ISTAT  FENOMENO        DATA  \
0            1   NaN         TORINO  -99.0      1272     -99.0  2012-11-02   
1            2   NaN       VERCELLI  -99.0      2158     -99.0  2012-11-02   
2            3   NaN         NOVARA  -99.0      3106     -99.0  2012-11-02   
3            4   NaN          CUNEO  -99.0      4078     -99.0  2012-11-02   
4            5   NaN           ASTI  -99.0      5005     -99.0  2012-11-02   
...        ...   ...            ...    ...       ...       ...         ...   
3157       102   NaN  VIBO VALENTIA  -99.0    102047     -99.0  2020-04-01   
3158       103   NaN       VERBANIA  -99.0    103072     -99.0  2020-04-01   
3159       108   NaN          MONZA  -99.0    108033     -99.0  2020-04-01   
3160       109   NaN          FERMO  -99.0    109006     -99.0  2020-04-01 

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Individuazione curve di temperatura minima o massima
Procedura terminata.


In [28]:
# df = pd.read_csv('s3://zus-prod-s3/preprocessato/sistema/temperatura_norm/zeus/storico/T_x_OSS_new.csv')
# df.drop("Unnamed: 0",axis=1,inplace=True)
# df.to_csv('s3://zus-prod-s3/preprocessato/sistema/temperatura_norm/zeus/storico/T_x_OSS_new.csv',index=False)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
pd.read_csv('s3://zus-prod-s3/preprocessato/utente/temperatura_norm/zeus/min_periodo_prov/best/min_periodo_11_3_2000_2020.csv')

,date,provincia,T_min,to_groupby,monthday
0,2011-11-01,84,15,2012,11-01
1,2011-11-02,84,15,2012,11-02
2,2011-11-03,84,13,2012,11-03
3,2011-11-04,84,16,2012,11-04
4,2011-11-05,84,14,2012,11-05
...,...,...,...,...,...
15548,2006-03-27,56,7,2006,03-27
15549,2006-03-28,56,5,2006,03-28
15550,2006-03-29,56,5,2006,03-29
15551,2006-03-30,56,4,2006,03-30


In [15]:
pd.read_csv('s3://zus-prod-s3/metadati/sistema/temperatura_norm/zeus/metadati/max_periodo_oss/200011_2020320210507153520/metadati.csv')

,CATEGORIA,FLUSSO,TIMESTAMP,ID,PATH,NOME_PARAMETRO,VALORE_PARAMETRO
0,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,FILE_INPUT,s3://zus-prod-s3/preprocessato/sistema/tempera...
1,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,INIZIO,200011
2,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,FINE,20203
3,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,MESE_INIZIO,11
4,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,MESE_FINE,3
5,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,ANNO_INIZIO,2000
6,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,ANNO_FINE,2020
7,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,COLONNA,max
8,TEMPERATURA_NORM,max_PERIODO_OSS,20210507153520,TEMPERATURA_NORM_max_PERIODO_OSS_20210507153520,./max_periodo_oss/200011_2020320210507153520/m...,PATH_TO_OUTPUT,s3://zus-prod-s3/./max_periodo_prov/best/max_p...


In [75]:
q=pd.date_range(str(2000)+'-'+str(11),str(2001)+'-'+str(3).rjust(2,'0'),freq='MS').strftime('%m').tolist()
q

['11', '12', '01', '02', '03']

In [63]:
str(3).rjust(2,'0')

'03'

In [79]:
df = costruzione_file_da_mensile('preprocessato/sistema/temperatura/epson/temperatura',2000,2020,11,3)

df

200001
200002
200003


,PROVINCIA,AREA,CAPOLUOGO,CIELO,COD_ISTAT,FENOMENO,DATA,DATA_VAL,TIPO,LATITUDINE,LONGITUDINE,TEMPERATURA_MAX,TEMPERATURA_MIN,UDM,QUOTA,REGIONE,DATA_AGGIORNAMENTO,OSSERVATORIO
0,1,NaN,TORINO,-99.0,1272,-99.0,2012-11-02,2012-11-01,1,45.06,7.68,15.0,4.0,°C,239,NaN,2021-05-10 15:41:48,11.0
1,2,NaN,VERCELLI,-99.0,2158,-99.0,2012-11-02,2012-11-01,1,45.32,8.42,12.0,7.0,°C,130,NaN,2021-05-10 15:41:48,NaN
2,3,NaN,NOVARA,-99.0,3106,-99.0,2012-11-02,2012-11-01,1,45.44,8.61,12.0,7.0,°C,162,NaN,2021-05-10 15:41:48,NaN
3,4,NaN,CUNEO,-99.0,4078,-99.0,2012-11-02,2012-11-01,1,44.49,7.54,10.0,1.0,°C,534,NaN,2021-05-10 15:41:48,NaN
4,5,NaN,ASTI,-99.0,5005,-99.0,2012-11-02,2012-11-01,1,44.89,8.20,13.0,6.0,°C,123,NaN,2021-05-10 15:41:48,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3157,102,NaN,VIBO VALENTIA,-99.0,102047,-99.0,2020-04-01,2020-03-31,1,38.67,16.10,12.0,10.0,°C,476,NaN,2021-05-10 15:41:48,NaN
3158,103,NaN,VERBANIA,-99.0,103072,-99.0,2020-04-01,2020-03-31,1,45.92,8.54,5.0,5.0,°C,197,NaN,2021-05-10 15:41:48,NaN
3159,108,NaN,MONZA,-99.0,108033,-99.0,2020-04-01,2020-03-31,1,45.58,9.27,11.0,4.0,°C,162,NaN,2021-05-10 15:41:48,NaN
3160,109,NaN,FERMO,-99.0,109006,-99.0,2020-04-01,2020-03-31,1,43.16,13.71,7.0,7.0,°C,319,NaN,2021-05-10 15:41:48,NaN
